# **Интерпретатор bf**
Можете попробовать сделать его сами, а потом поделиться результатами.  
[Что такое bf.](http://ru.wikipedia.org/wiki/Brainfuck)

In [1]:
from enum import Enum
    
class Ops(Enum):
    SHIFT_L = 1
    SHIFT_R = 2
    PLUS = 3
    MINUS = 4
    LABEL = 5
    GOTO = 6
    PRNT = 7
    INPT = 8
    EMPTY = 9

In [2]:
class Cells:
    
    n = 32_768
    
    max_val = 128
    
    def __init__(self):
        self.cells = [ 0 for i in range(self.n) ]
        self.pointer = 0
    
    def pointer_zero(self):
        return self.cells[self.pointer] == 0
    
    def pointer_not_zero(self):
        return self.cells[self.pointer] != 0
    
    def shift_r(self):
        self.pointer += 1
    
    def shift_l(self):
        self.pointer -= 1

    def plus(self):
        val = self.cells[self.pointer]
        if val >= self.max_val:
            self.cells[self.pointer] = -(self.max_val-1)
        else:
            self.cells[self.pointer] += 1
        
       
    def minus(self):
        val = self.cells[self.pointer]
        if val <= -self.max_val:
            self.cells[self.pointer] = (self.max_val-1)
        else:
            self.cells[self.pointer] -= 1
       
    def inpt(self):
        val = input("bf:>")
        self.cells[self.pointer] = int(val)
    
    def prnt(self):
        return self.cells[self.pointer]
    
    def reset(self):
        del self.cells
        self.cells = [0 for i in range(self.n)]
        self.pointer = 0

In [3]:
class SCycle:
    
    idx = 0
    goto = 0
        
class ECycle:
    
    goto = 0

In [4]:
actions = {
    Ops.PLUS : lambda tape : (tape.plus(), None),
    Ops.MINUS : lambda tape : (tape.minus(), None),
    Ops.SHIFT_R : lambda tape : (tape.shift_r(), None),
    Ops.SHIFT_L : lambda tape : (tape.shift_l(), None),
    Ops.INPT : lambda tape : (tape.inpt(), None),
    Ops.PRNT : lambda tape : (tape.prnt(), None),
    Ops.LABEL : lambda label : lambda tape : (None, label) if tape.pointer_zero() else (None, None),
    Ops.GOTO : lambda label : lambda tape : (None, label) if tape.pointer_not_zero() else (None, None),
}

In [5]:
grammar = {
    '+' : Ops.PLUS,
    '-' : Ops.MINUS,
    '>' : Ops.SHIFT_R,
    '<' : Ops.SHIFT_L,
    ',' : Ops.INPT,
    '.' : Ops.PRNT,
    '[' : Ops.LABEL,
    ']' : Ops.GOTO,
}

def bf_token(mc):
    return grammar.get(mc)

def bf_parse(s):
    return list(filter(lambda x : x in grammar, s))

In [6]:
def bf_read(tokens):
    
    tape = []
    
    cycles = []
    
    for idx,i in enumerate(tokens):
        
        token = bf_token(i)
        
        if i == '[':
            sc = SCycle()
            sc.idx = idx
            cycles.append(sc)
            
            tape.append((token, None))
            
            continue
            
        elif i == ']':
            
            sc = cycles.pop()
            ec = ECycle()
            ec.goto = sc.idx
            sc.goto = idx
                
            tape[ec.goto] = (sc.idx, Ops.LABEL, actions[Ops.LABEL](idx), sc.goto)
            tape.append((idx, token, actions[token](ec.goto), ec.goto))
            
            continue
        
        tape.append((idx, token, actions[token]))
    
    return tape

In [7]:
def bf_run(tape, instructions):
    
    pointer = 0
    
    while pointer < len(instructions):
        
        num, token, instruction, *_ = instructions[pointer]
        
        val, label = instruction(tape)
                
        if val:
            yield val
        elif label:
            pointer = label
            continue
        pointer += 1

In [8]:
prog = '''
>>>>>,>++<<<<<<>[-]>[-]>[-]>[-]>[<<<<+>>>>-]<<<<[>>>>>[
<<<<+>+>>>-]<<<[>>>+<<<-]<[>+<<-[>>[-]>+<<<-]>>>[<<<+>>
>-]<[<-[>>>-<<<[-]]+>-]<-]>>>+<<<<]>>>>>>+<<>>>+++++++[
>+++++<-]>---<<<<>>>+++++++[>>++++++<<-]<<<>>>++++++++++
<<<[->>>>>>+>+>>>>>>>>>+<<<<<<<<<<<<<<<<]>>[->>>>>>>+<<
<<<<<]>>>>+[>[->+>>>+>>+<<<<<<]>>[->+>>+<<<]>>[-<<<<<<<
.>>>>>>>]>[-<<<<<<<.>>>>>>>]>[-<<<<<<<<<.>>>>>>>>>]<<<<
<<<<<<.>>>>>>>>>><<<<<[-<+>]<->>>[-<+>]<++<<<-]>>>>>>>>
>>-[<<<<<<<<<<<<.>>>>>>>>>>>>-]<<<<<<<<<<<...

'''

In [9]:
quine = '''
->+>+++>>+>++>+>+++>>+>++>>>+>+>+>++>+>>>>+++>+>>++>+>+++>>++>++>>+>>+>++>++>+>>>>+++>+>>>>++>++>>>>+>>++>+>+++>>>++>>++++++>>+>>++>+>>>>+++>>+++++>>+>+++>>>++>>++>>+>>++>+>+++>>>++>>+++++++++++++>>+>>++>+>+++>+>+++>>>++>>++++>>+>>++>+>>>>+++>>+++++>>>>++>>>>+>+>++>>+++>+>>>>+++>+>>>>+++>+>>>>+++>>++>++>+>+++>+>++>++>>>>>>++>+>+++>>>>>+++>>>++>+>+++>+>+>++>>>>>>++>>>+>>>++>+>>>>+++>+>>>+>>++>+>++++++++++++++++++>>>>+>+>>>+>>++>+>+++>>>++>>++++++++>>+>>++>+>>>>+++>>++++++>>>+>++>>+++>+>+>++>+>+++>>>>>+++>>>+>+>>++>+>+++>>>++>>++++++++>>+>>++>+>>>>+++>>++++>>+>+++>>>>>>++>+>+++>>+>++>>>>+>+>++>+>>>>+++>>+++>>>+[[->>+<<]<+]+++++[->+++++++++<]>.[+]>>[<<+++++++[->+++++++++<]>-.------------------->-[-<.<+>>]<[+]<+>>>]<<<[-[-[-[>>+<++++++[->+++++<]]>++++++++++++++<]>+++<]++++++[->+++++++<]>+<<<-[->>>++<<<]>[->>.<<]<<]
'''

In [10]:
prog1 = '''
>++++[<++++++++>-]>++++++[<++++++++>-]<->>++++++++
[<++++++++++++>-]<---->++++++++++>++++++++++[>+>+<
<-]>>[<<+>>-]<[[>+>+<<-]>>[<<+>>-]<[<<<<<<.>>>>>>-
]<[>+>+<<-]>>[<<+>>-]<<<[>>>+>+<<<<-]>>>>[<<<<+>>>
>-]<<[>-<-]>+[<<<<<<.>.>>>>>-]<<<.>-]

'''

In [11]:
tape = Cells()

tk = bf_parse(prog)
instructions = bf_read(tk)
for c in bf_run(tape, instructions):
    print(chr(c), end='')

bf:> 23


           *           
          ***          
         *****         
        *******        
       *********       
      ***********      
     *************     
    ***************    
   *****************   
  *******************  
 ********************* 
***********************
          ***

# **С Наступающим!**

###### 